![A soccer pitch for an international match.](soccer-pitch.jpg)

You're working as a sports journalist at a major online sports media company, specializing in soccer analysis and reporting. You've been watching both men's and women's international soccer matches for a number of years, and your gut instinct tells you that more goals are scored in women's international football matches than men's. This would make an interesting investigative article that your subscribers are bound to love, but you'll need to perform a valid statistical hypothesis test to be sure!

While scoping this project, you acknowledge that the sport has changed a lot over the years, and performances likely vary a lot depending on the tournament, so you decide to limit the data used in the analysis to only official `FIFA World Cup` matches (not including qualifiers) after `2002-01-01`.

You create two datasets containing the results of every official men's and women's international football match since the 19th century, which you scraped from a reliable online source. This data is stored in two CSV files: `women_results.csv` and `men_results.csv`.

The question you are trying to determine the answer to is:

> Are more goals scored in women's international soccer matches than men's?

You assume a **10% significance level**, and use the following null and alternative hypotheses:

$H_0$ : The mean number of goals scored in women's international soccer matches is the same as men's.

$H_A$ : The mean number of goals scored in women's international soccer matches is greater than men's.

In [19]:
# Import necessary libraries
library(tidyverse)

In [20]:
# Load data
women_data <- read.csv("women_results.csv")
men_data <- read.csv("men_results.csv")

# Convert date columns to Date type
women_data$date <- as.Date(women_data$date)
men_data$date <- as.Date(men_data$date)

# Filter for FIFA World Cup matches after 2002-01-01
women_wc <- subset(women_data, tournament == "FIFA World Cup" & date >= as.Date("2002-01-01"))
men_wc <- subset(men_data, tournament == "FIFA World Cup" & date >= as.Date("2002-01-01"))

# Calculate total goals per match
women_goals <- women_wc$home_score + women_wc$away_score
men_goals <- men_wc$home_score + men_wc$away_score

# Check normality (Shapiro-Wilk test)
shapiro_women <- shapiro.test(women_goals)
shapiro_men <- shapiro.test(men_goals)

# Choose one-tailed test based on normality
if (shapiro_women$p.value > 0.05 & shapiro_men$p.value > 0.05) {
  # Use Welch's t-test (greater alternative)
  test <- t.test(women_goals, men_goals, var.equal = FALSE, alternative = "greater")
} else {
  # Use Wilcoxon rank-sum test (greater alternative)
  test <- wilcox.test(women_goals, men_goals, alternative = "greater")
}

# Extract p-value
p_val <- test$p.value

# 10% significance level decision
result <- ifelse(p_val < 0.10, "reject", "fail to reject")

# Store result
result_df <- data.frame(p_val = p_val, result = result)

# Output
print(result_df)


       p_val result
1 0.00510661 reject
